# Análisis de Wavelets para Datos MI-EEG

Este notebook implementa análisis avanzado de wavelets para extraer características temporales y espectrales de los datos EEG de imaginación motora.

## Objetivos

1. **Transformada Wavelet Continua (CWT)**: Análisis tiempo-frecuencia usando wavelet de Morlet
2. **Transformada Wavelet Discreta (DWT)**: Descomposición multiresolución usando Daubechies 4
3. **Extracción de características**: Energía por banda, frecuencia dominante, entropía espectral
4. **Preparación para BoF**: Características optimizadas para Bag of Features

## Pipeline

- Carga de datos EEG procesados
- Análisis CWT con escalas logarítmicas
- Análisis DWT con múltiples niveles
- Extracción de características estadísticas
- Visualizaciones tiempo-frecuencia
- Guardado de características para BoF


In [1]:
# Celda de instalación de dependencias
# Ejecutar esta celda SOLO si necesitas instalar las librerías en un entorno nuevo

%pip install mne
%pip install PyWavelets
%pip install scikit-learn
%pip install matplotlib
%pip install pandas
%pip install numpy
%pip install scipy
%pip install tqdm

print("Instalación de dependencias completada")



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


UnboundLocalError: cannot access local variable 'child' where it is not associated with a value

In [2]:
# Importar librerías necesarias
import json
import os
import re
from glob import glob
from pathlib import Path
from typing import Dict, List, Tuple, Optional

import matplotlib.pyplot as plt
import mne
import numpy as np
import pandas as pd
import pywt
from mne.io import read_raw_eeglab
from scipy.signal import welch
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

# Configurar matplotlib
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12
plt.style.use('seaborn-v0_8')

print("Librerías importadas correctamente")
print(f"PyWavelets versión: {pywt.__version__}")


Librerías importadas correctamente
PyWavelets versión: 1.8.0


## Carga de Datos del EDA

Este notebook continúa desde el análisis EDA previo. Cargamos los datos ya procesados para evitar duplicar el trabajo:


In [3]:
# Cargar datos procesados del EDA
shared_data_dir = Path("shared_data")

if not shared_data_dir.exists():
    print("Error: Directorio shared_data no encontrado.")
    print("Por favor ejecuta primero el notebook 01_EDA_Analysis.ipynb")
    raise FileNotFoundError("shared_data directory not found")

print("Cargando datos procesados del EDA...")

# Cargar datos principales
X = np.load(shared_data_dir / "X_data.npy")  # Datos concatenados (trials, channels, time)
ch_names = np.load(shared_data_dir / "ch_names.npy")  # Nombres de canales
sfreq = float(np.load(shared_data_dir / "sfreq.npy")[0])  # Frecuencia de muestreo
N, C, T = np.load(shared_data_dir / "data_dimensions.npy")  # Dimensiones

# Cargar información de sujetos
subjects_info_df = pd.read_csv(shared_data_dir / "subjects_info.csv")
subjects_info = subjects_info_df.to_dict('records')

# Cargar información de regiones
with open(shared_data_dir / "region_info.json", 'r') as f:
    region_info = json.load(f)
region_indices = region_info['region_indices']
REGION_PREFIXES = region_info['region_prefixes']

# Cargar parámetros de configuración
with open(shared_data_dir / "config_params.json", 'r') as f:
    config_params = json.load(f)
LOW_BAND = config_params['LOW_BAND']
HIGH_BAND = config_params['HIGH_BAND']
MU_BAND = tuple(config_params['MU_BAND'])
BETA_BAND = tuple(config_params['BETA_BAND'])
EXPECTED_TRIAL_SEC = config_params['EXPECTED_TRIAL_SEC']
CANDIDATE_DIRS = config_params['CANDIDATE_DIRS']

print("Datos cargados exitosamente:")
print(f"  - Datos concatenados: {X.shape} (trials, channels, time)")
print(f"  - Canales: {len(ch_names)}")
print(f"  - Frecuencia de muestreo: {sfreq} Hz")
print(f"  - Dimensiones: {N} trials, {C} canales, {T} muestras")
print(f"  - Duración por trial: {T/sfreq:.1f} segundos")
print(f"  - Sujetos procesados: {len(subjects_info)}")
print(f"  - Regiones identificadas: {list(region_indices.keys())}")


Cargando datos procesados del EDA...
Datos cargados exitosamente:
  - Datos concatenados: (880, 64, 1152) (trials, channels, time)
  - Canales: 64
  - Frecuencia de muestreo: 128.0 Hz
  - Dimensiones: 880 trials, 64 canales, 1152 muestras
  - Duración por trial: 9.0 segundos
  - Sujetos procesados: 40
  - Regiones identificadas: ['F', 'FC', 'C', 'CP', 'P', 'PO', 'O']


In [4]:
# Configuración de directorios
data_root = Path(".").resolve()  # Directorio actual
output_dir = Path("reports")
wavelet_output_dir = Path("wavelet_reports")
wavelet_output_dir.mkdir(parents=True, exist_ok=True)

print(f"\nConfiguración de directorios:")
print(f"  - Directorio de datos: {data_root}")
print(f"  - Directorio de salida EDA: {output_dir.resolve()}")
print(f"  - Directorio de salida wavelets: {wavelet_output_dir.resolve()}")
print(f"  - Directorio de datos compartidos: {shared_data_dir.resolve()}")



Configuración de directorios:
  - Directorio de datos: /Users/manueljurado/Downloads/datos_BCI
  - Directorio de salida EDA: /Users/manueljurado/Downloads/datos_BCI/reports
  - Directorio de salida wavelets: /Users/manueljurado/Downloads/datos_BCI/wavelet_reports
  - Directorio de datos compartidos: /Users/manueljurado/Downloads/datos_BCI/shared_data


## Configuración de Wavelets

Definimos los parámetros para el análisis de wavelets:


In [5]:
# Configurar parámetros de wavelets
CWT_SCALES = np.logspace(0.5, 2.5, 50)  # Escalas logarítmicas (1.6-316 Hz aprox)
CWT_WAVELET = 'cmor5.0-1.0'  # Complex Morlet wavelet para análisis tiempo-frecuencia (ancho 5.0, frecuencia central 1.0)
CWT_WIDTH = 5.0  # Ancho de la wavelet de Morlet

# Parámetros DWT
DWT_WAVELET = 'db4'  # Wavelet Daubechies 4
DWT_LEVELS = 6  # Niveles de descomposición

# Bandas de frecuencia de interés para wavelets
FREQ_BANDS = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 100)
}

print("Configuración de wavelets:")
print(f"Escalas CWT: {len(CWT_SCALES)} escalas desde {CWT_SCALES[0]:.1f} hasta {CWT_SCALES[-1]:.1f}")
print(f"Wavelet CWT: {CWT_WAVELET} con ancho {CWT_WIDTH}")
print(f"Wavelet DWT: {DWT_WAVELET} con {DWT_LEVELS} niveles")
print(f"Bandas de frecuencia: {list(FREQ_BANDS.keys())}")


Configuración de wavelets:
Escalas CWT: 50 escalas desde 3.2 hasta 316.2
Wavelet CWT: cmor5.0-1.0 con ancho 5.0
Wavelet DWT: db4 con 6 niveles
Bandas de frecuencia: ['delta', 'theta', 'alpha', 'beta', 'gamma']


### Funciones Auxiliares para Wavelets

Definimos las funciones específicas para el análisis de wavelets:


In [6]:
def scales_to_frequencies(scales: np.ndarray, wavelet: str, sampling_rate: float) -> np.ndarray:
    """Convierte escalas wavelet a frecuencias."""
    if wavelet.startswith('cmor'):
        # Para Complex Morlet: freq = (width * sampling_rate) / (2 * pi * scale)
        # Extraer ancho de banda del nombre de la wavelet (formato: cmorB-C)
        if '-' in wavelet:
            bandwidth = float(wavelet.split('-')[0].replace('cmor', ''))
        else:
            bandwidth = CWT_WIDTH
        frequencies = (bandwidth * sampling_rate) / (2 * np.pi * scales)
    else:
        # Para otras wavelets, aproximación general
        frequencies = sampling_rate / (2 * scales)
    return frequencies

def compute_cwt_coefficients(data: np.ndarray, sfreq: float) -> Tuple[np.ndarray, np.ndarray]:
    """
    Calcula coeficientes CWT para todos los canales y trials.
    
    Args:
        data: Array de forma (trials, channels, time)
        sfreq: Frecuencia de muestreo
    
    Returns:
        Tuple de (coeficientes_cwt, frecuencias)
    """
    n_trials, n_channels, n_times = data.shape
    
    # Calcular frecuencias correspondientes a las escalas
    frequencies = scales_to_frequencies(CWT_SCALES, CWT_WAVELET, sfreq)
    
    # Inicializar array para coeficientes CWT
    cwt_coeffs = np.zeros((n_trials, n_channels, len(CWT_SCALES), n_times), dtype=complex)
    
    print("Calculando coeficientes CWT...")
    for trial_idx in tqdm(range(n_trials), desc="Trials"):
        for ch_idx in range(n_channels):
            signal = data[trial_idx, ch_idx, :]
            
            # Calcular CWT
            coefficients, _ = pywt.cwt(signal, CWT_SCALES, CWT_WAVELET, 
                                     sampling_period=1.0/sfreq)
            cwt_coeffs[trial_idx, ch_idx, :, :] = coefficients
    
    return cwt_coeffs, frequencies

print("Funciones auxiliares para wavelets definidas")


Funciones auxiliares para wavelets definidas


## Análisis CWT (Transformada Wavelet Continua)

Calculamos los coeficientes CWT para análisis tiempo-frecuencia:


In [7]:
# Calcular coeficientes CWT usando los datos concatenados
print("Iniciando análisis CWT...")
print(f"Datos de entrada: {X.shape} (trials, channels, time)")
print(f"Frecuencia de muestreo: {sfreq} Hz")

# Calcular CWT
cwt_coeffs, cwt_frequencies = compute_cwt_coefficients(X, sfreq)

print(f"\nCWT completado:")
print(f"Coeficientes CWT: {cwt_coeffs.shape}")
print(f"Frecuencias CWT: {len(cwt_frequencies)} puntos")
print(f"Rango de frecuencias: {cwt_frequencies[0]:.1f} - {cwt_frequencies[-1]:.1f} Hz")

# Mostrar información sobre las bandas de frecuencia
print(f"\nBandas de frecuencia CWT:")
for band_name, (fmin, fmax) in FREQ_BANDS.items():
    mask = (cwt_frequencies >= fmin) & (cwt_frequencies <= fmax)
    n_scales = mask.sum()
    if n_scales > 0:
        print(f"  - {band_name} ({fmin}-{fmax} Hz): {n_scales} escalas")
    else:
        print(f"  - {band_name} ({fmin}-{fmax} Hz): fuera del rango")


Iniciando análisis CWT...
Datos de entrada: (880, 64, 1152) (trials, channels, time)
Frecuencia de muestreo: 128.0 Hz
Calculando coeficientes CWT...


Trials: 100%|██████████| 880/880 [45:34<00:00,  3.11s/it]


CWT completado:
Coeficientes CWT: (880, 64, 50, 1152)
Frecuencias CWT: 50 puntos
Rango de frecuencias: 32.2 - 0.3 Hz

Bandas de frecuencia CWT:
  - delta (1-4 Hz): 14 escalas
  - theta (4-8 Hz): 8 escalas
  - alpha (8-13 Hz): 5 escalas
  - beta (13-30 Hz): 9 escalas
  - gamma (30-100 Hz): 1 escalas


### Extracción de Características CWT

Extraemos características estadísticas de los coeficientes CWT para BoF:


In [8]:
def extract_cwt_features(cwt_coeffs: np.ndarray, frequencies: np.ndarray) -> Dict[str, np.ndarray]:
    """
    Extrae características de los coeficientes CWT.
    
    Args:
        cwt_coeffs: Coeficientes CWT de forma (trials, channels, scales, time)
        frequencies: Array de frecuencias correspondientes
    
    Returns:
        Diccionario con características extraídas
    """
    n_trials, n_channels, n_scales, n_times = cwt_coeffs.shape
    
    features = {}
    
    print("Extrayendo características CWT...")
    
    # 1. Energía por banda de frecuencia
    for band_name, (fmin, fmax) in FREQ_BANDS.items():
        # Encontrar índices de escalas correspondientes a esta banda
        band_mask = (frequencies >= fmin) & (frequencies <= fmax)
        if not band_mask.any():
            continue
            
        # Energía promedio por trial y canal
        band_coeffs = cwt_coeffs[:, :, band_mask, :]
        energy = np.mean(np.abs(band_coeffs)**2, axis=(2, 3))  # (trials, channels)
        features[f'cwt_energy_{band_name}'] = energy
        print(f"  - Energía {band_name}: {energy.shape}")
    
    # 2. Potencia máxima por escala
    max_power_per_scale = np.max(np.abs(cwt_coeffs)**2, axis=3)  # (trials, channels, scales)
    features['cwt_max_power'] = max_power_per_scale
    print(f"  - Potencia máxima por escala: {max_power_per_scale.shape}")
    
    # 3. Frecuencia dominante por trial y canal
    power_spectrum = np.abs(cwt_coeffs)**2
    dominant_freq_idx = np.argmax(np.mean(power_spectrum, axis=3), axis=2)  # (trials, channels)
    dominant_frequencies = frequencies[dominant_freq_idx]
    features['cwt_dominant_freq'] = dominant_frequencies
    print(f"  - Frecuencia dominante: {dominant_frequencies.shape}")
    
    # 4. Entropía espectral
    spectral_entropy = np.zeros((n_trials, n_channels))
    for trial_idx in range(n_trials):
        for ch_idx in range(n_channels):
            power = np.mean(np.abs(cwt_coeffs[trial_idx, ch_idx, :, :])**2, axis=1)
            power_norm = power / np.sum(power)
            spectral_entropy[trial_idx, ch_idx] = -np.sum(power_norm * np.log(power_norm + 1e-10))
    features['cwt_spectral_entropy'] = spectral_entropy
    print(f"  - Entropía espectral: {spectral_entropy.shape}")
    
    return features

# Extraer características CWT
cwt_features = extract_cwt_features(cwt_coeffs, cwt_frequencies)

print(f"\nCaracterísticas CWT extraídas:")
for feature_name, feature_array in cwt_features.items():
    print(f"  - {feature_name}: {feature_array.shape}")


Extrayendo características CWT...


: 

## Análisis DWT (Transformada Wavelet Discreta)

Implementamos análisis DWT para descomposición multiresolución:


In [ ]:
def compute_dwt_coefficients(data: np.ndarray, wavelet: str, levels: int) -> Dict[str, np.ndarray]:
    """
    Calcula coeficientes DWT para todos los canales y trials.
    
    Args:
        data: Array de forma (trials, channels, time)
        wavelet: Tipo de wavelet (ej. 'db4')
        levels: Número de niveles de descomposición
    
    Returns:
        Diccionario con coeficientes DWT por nivel
    """
    n_trials, n_channels, n_times = data.shape
    
    # Inicializar diccionario para coeficientes
    dwt_coeffs = {}
    
    print("Calculando coeficientes DWT...")
    for trial_idx in tqdm(range(n_trials), desc="Trials"):
        for ch_idx in range(n_channels):
            signal = data[trial_idx, ch_idx, :]
            
            # Calcular DWT
            coeffs = pywt.wavedec(signal, wavelet, level=levels)
            
            # Guardar coeficientes por nivel
            for level, coeff in enumerate(coeffs):
                key = f'level_{level}'
                if key not in dwt_coeffs:
                    dwt_coeffs[key] = np.zeros((n_trials, n_channels, len(coeff)))
                dwt_coeffs[key][trial_idx, ch_idx, :] = coeff
    
    return dwt_coeffs

# Calcular coeficientes DWT
print("Iniciando análisis DWT...")
print(f"Datos de entrada: {X.shape} (trials, channels, time)")
print(f"Wavelet: {DWT_WAVELET}, Niveles: {DWT_LEVELS}")

dwt_coeffs = compute_dwt_coefficients(X, DWT_WAVELET, DWT_LEVELS)

print(f"\nDWT completado:")
print(f"Coeficientes DWT por nivel:")
for level_key, coeff_array in dwt_coeffs.items():
    print(f"  - {level_key}: {coeff_array.shape}")


### Extracción de Características DWT

Extraemos características estadísticas de los coeficientes DWT:


In [ ]:
def extract_dwt_features(dwt_coeffs: Dict[str, np.ndarray]) -> Dict[str, np.ndarray]:
    """
    Extrae características de los coeficientes DWT.
    
    Args:
        dwt_coeffs: Diccionario con coeficientes DWT por nivel
    
    Returns:
        Diccionario con características extraídas
    """
    features = {}
    
    print("Extrayendo características DWT...")
    
    # 1. Energía por nivel de descomposición
    for level_key, coeff_array in dwt_coeffs.items():
        # Energía promedio por trial y canal
        energy = np.mean(coeff_array**2, axis=2)  # (trials, channels)
        features[f'dwt_energy_{level_key}'] = energy
        print(f"  - Energía {level_key}: {energy.shape}")
    
    # 2. Estadísticas por nivel
    for level_key, coeff_array in dwt_coeffs.items():
        # Media por trial y canal
        mean_coeffs = np.mean(coeff_array, axis=2)  # (trials, channels)
        features[f'dwt_mean_{level_key}'] = mean_coeffs
        
        # Desviación estándar por trial y canal
        std_coeffs = np.std(coeff_array, axis=2)  # (trials, channels)
        features[f'dwt_std_{level_key}'] = std_coeffs
        
        # Máximo absoluto por trial y canal
        max_coeffs = np.max(np.abs(coeff_array), axis=2)  # (trials, channels)
        features[f'dwt_max_{level_key}'] = max_coeffs
        
        print(f"  - Estadísticas {level_key}: media, std, max")
    
    # 3. Relación de energía entre niveles (aproximación de bandas de frecuencia)
    if 'level_0' in dwt_coeffs and 'level_1' in dwt_coeffs:
        # Relación entre aproximación y detalle
        energy_level_0 = np.mean(dwt_coeffs['level_0']**2, axis=2)
        energy_level_1 = np.mean(dwt_coeffs['level_1']**2, axis=2)
        energy_ratio = energy_level_0 / (energy_level_1 + 1e-10)
        features['dwt_energy_ratio_level0_level1'] = energy_ratio
        print(f"  - Relación de energía level0/level1: {energy_ratio.shape}")
    
    return features

# Extraer características DWT
dwt_features = extract_dwt_features(dwt_coeffs)

print(f"\nCaracterísticas DWT extraídas:")
for feature_name, feature_array in dwt_features.items():
    print(f"  - {feature_name}: {feature_array.shape}")


## Preparación de Características para BoF

Combinamos todas las características extraídas y las preparamos para Bag of Features:


In [ ]:
# Combinar todas las características
print("Combinando características para BoF...")

# Combinar características CWT y DWT
all_features = {}
all_features.update(cwt_features)
all_features.update(dwt_features)

print(f"Total de características extraídas: {len(all_features)}")

# Crear matriz de características para BoF
feature_names = []
feature_matrices = []

for feature_name, feature_array in all_features.items():
    # Reshape para tener una matriz 2D (samples, features)
    if len(feature_array.shape) == 2:  # (trials, channels)
        # Cada canal es una característica
        for ch_idx, ch_name in enumerate(ch_names):
            feature_names.append(f"{feature_name}_{ch_name}")
            feature_matrices.append(feature_array[:, ch_idx])
    elif len(feature_array.shape) == 3:  # (trials, channels, scales/levels)
        # Cada combinación canal-escala es una característica
        for ch_idx, ch_name in enumerate(ch_names):
            for scale_idx in range(feature_array.shape[2]):
                feature_names.append(f"{feature_name}_{ch_name}_scale{scale_idx}")
                feature_matrices.append(feature_array[:, ch_idx, scale_idx])

# Crear matriz final de características
X_features = np.column_stack(feature_matrices)  # (trials, total_features)

print(f"Matriz de características final:")
print(f"  - Trials: {X_features.shape[0]}")
print(f"  - Características totales: {X_features.shape[1]}")
print(f"  - Primeras 10 características: {feature_names[:10]}")

# Normalizar características
scaler = StandardScaler()
X_features_normalized = scaler.fit_transform(X_features)

print(f"Características normalizadas: {X_features_normalized.shape}")
print(f"Media después de normalización: {X_features_normalized.mean():.6f}")
print(f"Desviación estándar después de normalización: {X_features_normalized.std():.6f}")


## Guardado de Archivos para BoF

Guardamos las características y metadatos necesarios para implementar Bag of Features:


In [ ]:
# Guardar características normalizadas para BoF
features_file = wavelet_output_dir / "wavelet_features.npy"
np.save(features_file, X_features_normalized)
print(f"Características guardadas: {features_file.resolve()}")

# Guardar nombres de características
feature_names_file = wavelet_output_dir / "feature_names.txt"
with open(feature_names_file, 'w') as f:
    for name in feature_names:
        f.write(f"{name}\n")
print(f"Nombres de características guardados: {feature_names_file.resolve()}")

# Guardar información de canales
channel_info = pd.DataFrame({
    'channel_index': range(len(ch_names)),
    'channel_name': ch_names,
    'region': ['unknown'] * len(ch_names)  # Se puede mejorar con mapeo de regiones
})

# Mapear regiones basado en prefijos
for idx, ch_name in enumerate(ch_names):
    ch_upper = ch_name.upper()
    for region, prefixes in REGION_PREFIXES.items():
        if any(ch_upper.startswith(p) for p in prefixes):
            channel_info.loc[idx, 'region'] = region
            break

channel_info_file = wavelet_output_dir / "channel_info.csv"
channel_info.to_csv(channel_info_file, index=False)
print(f"Información de canales guardada: {channel_info_file.resolve()}")

# Guardar información de sujetos y tareas
subjects_df = pd.DataFrame(subjects_info)
subjects_file = wavelet_output_dir / "subjects_info.csv"
subjects_df.to_csv(subjects_file, index=False)
print(f"Información de sujetos guardada: {subjects_file.resolve()}")

# Guardar parámetros de configuración
config_info = {
    'cwt_scales': CWT_SCALES.tolist(),
    'cwt_wavelet': CWT_WAVELET,
    'cwt_width': CWT_WIDTH,
    'dwt_wavelet': DWT_WAVELET,
    'dwt_levels': DWT_LEVELS,
    'freq_bands': FREQ_BANDS,
    'sampling_rate': sfreq,
    'n_trials': N,
    'n_channels': C,
    'n_timepoints': T,
    'feature_dimensions': X_features_normalized.shape[1]
}

config_file = wavelet_output_dir / "wavelet_config.json"
import json
with open(config_file, 'w') as f:
    json.dump(config_info, f, indent=2)
print(f"Configuración guardada: {config_file.resolve()}")

print(f"\nResumen de archivos generados:")
print(f"  - {features_file.name}: Características normalizadas ({X_features_normalized.shape})")
print(f"  - {feature_names_file.name}: Nombres de características ({len(feature_names)} características)")
print(f"  - {channel_info_file.name}: Información de canales ({len(ch_names)} canales)")
print(f"  - {subjects_file.name}: Información de sujetos ({len(subjects_info)} archivos)")
print(f"  - {config_file.name}: Parámetros de configuración")


## Preparación de Datos para Bag of Features (BoF)

Esta sección prepara específicamente los datos que necesita el algoritmo Bag of Features para clasificación:


In [ ]:
# Crear directorio específico para datos de BoF
bof_data_dir = Path("bof_data")
bof_data_dir.mkdir(parents=True, exist_ok=True)

print("Preparando datos específicos para Bag of Features...")
print(f"Directorio BoF: {bof_data_dir.resolve()}")

# 1. Crear etiquetas de clase basadas en la tarea (left/right)
print("\n1. Creando etiquetas de clase...")

# Crear array de etiquetas basado en subjects_info
y_labels = []
trial_to_subject = []  # Mapeo de trial a sujeto
trial_to_task = []     # Mapeo de trial a tarea

trial_idx = 0
for subject_info in subjects_info:
    n_trials = subject_info['n_trials']
    task = subject_info['task']
    subject = subject_info['subject']
    
    # Etiquetas: 0 = left, 1 = right
    label = 0 if task == 'left' else 1
    
    for _ in range(n_trials):
        y_labels.append(label)
        trial_to_subject.append(subject)
        trial_to_task.append(task)
        trial_idx += 1

y_labels = np.array(y_labels)
print(f"  - Etiquetas creadas: {y_labels.shape}")
print(f"  - Clase 0 (left): {np.sum(y_labels == 0)} trials")
print(f"  - Clase 1 (right): {np.sum(y_labels == 1)} trials")

# Guardar etiquetas
np.save(bof_data_dir / "y_labels.npy", y_labels)
np.save(bof_data_dir / "trial_to_subject.npy", np.array(trial_to_subject))
np.save(bof_data_dir / "trial_to_task.npy", np.array(trial_to_task))

print(f"  - Etiquetas guardadas en: {bof_data_dir / 'y_labels.npy'}")


In [ ]:
# 2. Preparar características específicas para BoF
print("\n2. Preparando características para BoF...")

# Seleccionar características más relevantes para BoF
selected_features = {}

# Características CWT más importantes
cwt_key_features = [
    'cwt_energy_alpha',    # Energía en banda alpha (8-13 Hz)
    'cwt_energy_beta',    # Energía en banda beta (13-30 Hz)
    'cwt_dominant_freq',  # Frecuencia dominante
    'cwt_spectral_entropy' # Entropía espectral
]

for feature_name in cwt_key_features:
    if feature_name in cwt_features:
        selected_features[feature_name] = cwt_features[feature_name]
        print(f"  - {feature_name}: {cwt_features[feature_name].shape}")

# Características DWT más importantes (primeros 3 niveles)
dwt_key_features = [
    'dwt_energy_level_0',  # Aproximación (baja frecuencia)
    'dwt_energy_level_1',  # Primer detalle
    'dwt_energy_level_2',  # Segundo detalle
    'dwt_mean_level_0',    # Media de aproximación
    'dwt_std_level_0'      # Desviación estándar de aproximación
]

for feature_name in dwt_key_features:
    if feature_name in dwt_features:
        selected_features[feature_name] = dwt_features[feature_name]
        print(f"  - {feature_name}: {dwt_features[feature_name].shape}")

print(f"\nTotal de características seleccionadas: {len(selected_features)}")

# Crear matriz de características seleccionadas
bof_feature_names = []
bof_feature_matrices = []

for feature_name, feature_array in selected_features.items():
    if len(feature_array.shape) == 2:  # (trials, channels)
        for ch_idx, ch_name in enumerate(ch_names):
            bof_feature_names.append(f"{feature_name}_{ch_name}")
            bof_feature_matrices.append(feature_array[:, ch_idx])
    elif len(feature_array.shape) == 3:  # (trials, channels, scales)
        for ch_idx, ch_name in enumerate(ch_names):
            for scale_idx in range(feature_array.shape[2]):
                bof_feature_names.append(f"{feature_name}_{ch_name}_scale{scale_idx}")
                bof_feature_matrices.append(feature_array[:, ch_idx, scale_idx])

# Crear matriz final de características BoF
X_bof = np.column_stack(bof_feature_matrices)

print(f"Matriz de características BoF:")
print(f"  - Trials: {X_bof.shape[0]}")
print(f"  - Características: {X_bof.shape[1]}")
print(f"  - Primeras 10 características: {bof_feature_names[:10]}")

# Normalizar características BoF
scaler_bof = StandardScaler()
X_bof_normalized = scaler_bof.fit_transform(X_bof)

print(f"Características BoF normalizadas: {X_bof_normalized.shape}")
print(f"Media después de normalización: {X_bof_normalized.mean():.6f}")
print(f"Desviación estándar: {X_bof_normalized.std():.6f}")


In [ ]:
# 3. Guardar datos específicos para BoF
print("\n3. Guardando datos específicos para BoF...")

# Guardar características BoF
np.save(bof_data_dir / "X_bof_features.npy", X_bof_normalized)
print(f"  - Características BoF: {bof_data_dir / 'X_bof_features.npy'}")

# Guardar nombres de características BoF
with open(bof_data_dir / "bof_feature_names.txt", 'w') as f:
    for name in bof_feature_names:
        f.write(f"{name}\n")
print(f"  - Nombres de características: {bof_data_dir / 'bof_feature_names.txt'}")

# Guardar normalizador BoF
import pickle
with open(bof_data_dir / "scaler_bof.pkl", 'wb') as f:
    pickle.dump(scaler_bof, f)
print(f"  - Normalizador BoF: {bof_data_dir / 'scaler_bof.pkl'}")

# Crear información de metadatos para BoF
bof_metadata = {
    'n_trials': X_bof_normalized.shape[0],
    'n_features': X_bof_normalized.shape[1],
    'n_channels': len(ch_names),
    'n_subjects': len(subjects_info),
    'class_distribution': {
        'left_trials': int(np.sum(y_labels == 0)),
        'right_trials': int(np.sum(y_labels == 1))
    },
    'feature_types': {
        'cwt_features': len([f for f in bof_feature_names if f.startswith('cwt_')]),
        'dwt_features': len([f for f in bof_feature_names if f.startswith('dwt_')])
    },
    'sampling_rate': sfreq,
    'trial_duration_sec': T / sfreq,
    'wavelet_config': {
        'cwt_scales': len(CWT_SCALES),
        'cwt_wavelet': CWT_WAVELET,
        'dwt_levels': DWT_LEVELS,
        'dwt_wavelet': DWT_WAVELET
    }
}

with open(bof_data_dir / "bof_metadata.json", 'w') as f:
    json.dump(bof_metadata, f, indent=2)
print(f"  - Metadatos BoF: {bof_data_dir / 'bof_metadata.json'}")

# Crear archivo de configuración para BoF
bof_config = {
    'data_files': {
        'features': 'X_bof_features.npy',
        'labels': 'y_labels.npy',
        'feature_names': 'bof_feature_names.txt',
        'scaler': 'scaler_bof.pkl',
        'metadata': 'bof_metadata.json'
    },
    'recommended_params': {
        'n_clusters': [50, 100, 200],  # Número de clusters para BoF
        'random_state': 42,
        'test_size': 0.2,
        'cv_folds': 5
    },
    'feature_info': {
        'total_features': len(bof_feature_names),
        'cwt_features': len([f for f in bof_feature_names if f.startswith('cwt_')]),
        'dwt_features': len([f for f in bof_feature_names if f.startswith('dwt_')]),
        'normalized': True,
        'scaler_type': 'StandardScaler'
    }
}

with open(bof_data_dir / "bof_config.json", 'w') as f:
    json.dump(bof_config, f, indent=2)
print(f"  - Configuración BoF: {bof_data_dir / 'bof_config.json'}")

print(f"\nDatos BoF guardados exitosamente en: {bof_data_dir.resolve()}")


## Resumen del Análisis de Wavelets y Preparación BoF

### Análisis Completados

1. **Transformada Wavelet Continua (CWT)**
   - Wavelet de Morlet con 50 escalas logarítmicas
   - Análisis tiempo-frecuencia completo
   - Extracción de energía por banda, frecuencia dominante y entropía espectral

2. **Transformada Wavelet Discreta (DWT)**
   - Wavelet Daubechies 4 con 6 niveles de descomposición
   - Análisis multiresolución
   - Extracción de estadísticas por nivel (energía, media, desviación estándar, máximo)

3. **Preparación Específica para BoF**
   - Selección de características más relevantes
   - Normalización específica para BoF
   - Etiquetas de clase (left/right)
   - Metadatos completos y configuración

### Archivos Generados para BoF

Todos los archivos específicos para BoF se guardaron en el directorio `bof_data/`:

- **`X_bof_features.npy`**: Características normalizadas específicas para BoF
- **`y_labels.npy`**: Etiquetas de clase (0=left, 1=right)
- **`bof_feature_names.txt`**: Nombres de características seleccionadas
- **`scaler_bof.pkl`**: Normalizador entrenado para BoF
- **`bof_metadata.json`**: Metadatos completos del dataset
- **`bof_config.json`**: Configuración y parámetros recomendados
- **`trial_to_subject.npy`**: Mapeo de trials a sujetos
- **`trial_to_task.npy`**: Mapeo de trials a tareas

### Características Seleccionadas para BoF

**CWT Features:**
- Energía en banda alpha (8-13 Hz)
- Energía en banda beta (13-30 Hz)
- Frecuencia dominante por canal
- Entropía espectral por canal

**DWT Features:**
- Energía de aproximación (nivel 0)
- Energía de detalles (niveles 1-2)
- Media y desviación estándar de aproximación

### Próximos Pasos para BoF

Los datos están completamente preparados para implementar Bag of Features:

1. **Cargar datos**: Usar archivos en `bof_data/`
2. **Clustering**: Aplicar K-means con parámetros recomendados
3. **Codificación**: Crear histogramas de características por trial
4. **Clasificación**: Entrenar clasificadores SVM/Random Forest
5. **Evaluación**: Validación cruzada y métricas de rendimiento

### Variables Disponibles para BoF

- `X_bof_normalized`: Características normalizadas para BoF
- `y_labels`: Etiquetas de clase
- `bof_feature_names`: Nombres de características
- `scaler_bof`: Normalizador entrenado
- `bof_metadata`: Metadatos del dataset
- `bof_config`: Configuración recomendada
